In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.proportion import proportions_ztest
import statsmodels.api as sm
import math

In [2]:
df = pd.read_excel("/Users/kryptonempyrean/Desktop/Tesi Material/OneDrive_1_08-09-2024/Psoriasis_2017_Erez_Data2_Coded codifica in corso.xlsx", sheet_name="PsoriasisPanel")

In [3]:
df = df.iloc[:280450]

In [4]:
columns = pd.read_excel("/Users/kryptonempyrean/Desktop/Tesi Material/OneDrive_1_08-09-2024/Psoriasis_2017_Erez_Data2_Coded codifica in corso.xlsx", sheet_name="Foglio1", header=None)
columns = columns[1].tolist()

comorbidities = columns[:142]
selected_comorbidities = sorted(set(df.columns.tolist()) & set(comorbidities),  key = df.columns.tolist().index)
selected_comorbidities.remove("Psoriasis")

selected_columns = ['date_of_birth','GroupName','age','sex','Start_D','DiagYear', "Infertility ", "Psoriasis"] + selected_comorbidities

In [26]:
data = df[selected_columns]
data

,date_of_birth,GroupName,age,sex,Start_D,DiagYear,Infertility,Psoriasis,Tuberculosis,Tuberculosis s/p,...,OncBone,OncSaracoma,OncGenitalia,Oncmyeloma,OncPolycythemiaVera,OncMyelodysplastic,OncLympholiferative,OncNeurofibromatosis,OncOther,OncUnKnow
0,1906-05-15 00:00:00,Psoriasis,98,M,2005-08-03,2005.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1906-06-15 00:00:00,Control,98,M,2005-08-03,2005.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1907-06-15 00:00:00,Psoriasis,95,M,2003-10-12,2003.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1907-05-14 00:00:00,Control,95,M,2003-10-12,2003.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1909-01-01 00:00:00,Psoriasis,100,M,2009-06-30,2009.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280445,2016-06-03 00:00:00,Psoriasis,0,F,2017-11-14,2017.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280446,2017-01-03 00:00:00,Control,0,F,2017-05-15,2017.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280447,2017-01-08 00:00:00,Psoriasis,0,F,2017-05-15,2017.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280448,2017-01-08 00:00:00,Control,0,F,2017-11-21,2017.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
data = data[data.GroupName == 'Psoriasis']
data = data[data.sex == 'M']

In [28]:
data

,date_of_birth,GroupName,age,sex,Start_D,DiagYear,Infertility,Psoriasis,Tuberculosis,Tuberculosis s/p,...,OncBone,OncSaracoma,OncGenitalia,Oncmyeloma,OncPolycythemiaVera,OncMyelodysplastic,OncLympholiferative,OncNeurofibromatosis,OncOther,OncUnKnow
0,1906-05-15 00:00:00,Psoriasis,98,M,2005-08-03,2005.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1907-06-15 00:00:00,Psoriasis,95,M,2003-10-12,2003.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1909-01-01 00:00:00,Psoriasis,100,M,2009-06-30,2009.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1909-05-01 00:00:00,Psoriasis,93,M,2003-04-24,2003.0,NaN,1999-11-16 00:00:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1909-06-09 00:00:00,Psoriasis,94,M,2004-09-20,2004.0,NaN,2004-02-23 00:00:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138674,2016-03-15 00:00:00,Psoriasis,0,M,2017-03-14,2017.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138676,2016-01-29 00:00:00,Psoriasis,1,M,2017-01-02,2017.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138679,2016-01-30 00:00:00,Psoriasis,1,M,2017-03-30,2017.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138681,2016-06-15 00:00:00,Psoriasis,0,M,2017-02-28,2017.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Converting date columns to datetime for comparison
data['Psoriasis'] = pd.to_datetime(data['Psoriasis'], errors='coerce')
data['Start_D'] = pd.to_datetime(data['Start_D'], errors='coerce')

# Set Psoriasis date based on group and availability
# If 'Psoriasis' is NaT and the group is "Psoriasis", use 'Start_D' as the proxy
data['Psoriasis'] = data.apply(
    lambda row: row['Start_D'] if pd.isna(row['Psoriasis']) and row['GroupName'] == 'Psoriasis' else row['Psoriasis'],
    axis=1
)

In [30]:
data

,date_of_birth,GroupName,age,sex,Start_D,DiagYear,Infertility,Psoriasis,Tuberculosis,Tuberculosis s/p,...,OncBone,OncSaracoma,OncGenitalia,Oncmyeloma,OncPolycythemiaVera,OncMyelodysplastic,OncLympholiferative,OncNeurofibromatosis,OncOther,OncUnKnow
0,1906-05-15 00:00:00,Psoriasis,98,M,2005-08-03,2005.0,NaN,2005-08-03,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1907-06-15 00:00:00,Psoriasis,95,M,2003-10-12,2003.0,NaN,2003-10-12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1909-01-01 00:00:00,Psoriasis,100,M,2009-06-30,2009.0,NaN,2009-06-30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1909-05-01 00:00:00,Psoriasis,93,M,2003-04-24,2003.0,NaN,1999-11-16,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1909-06-09 00:00:00,Psoriasis,94,M,2004-09-20,2004.0,NaN,2004-02-23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138674,2016-03-15 00:00:00,Psoriasis,0,M,2017-03-14,2017.0,NaN,2017-03-14,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138676,2016-01-29 00:00:00,Psoriasis,1,M,2017-01-02,2017.0,NaN,2017-01-02,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138679,2016-01-30 00:00:00,Psoriasis,1,M,2017-03-30,2017.0,NaN,2017-03-30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138681,2016-06-15 00:00:00,Psoriasis,0,M,2017-02-28,2017.0,NaN,2017-02-28,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# Convert Infertility column to datetime for comparison
data['Infertility '] = pd.to_datetime(data['Infertility '], errors='coerce')

# Filter patients who have infertility diagnosed after psoriasis or have NaN under infertility
data = data[
    (data['Infertility '].isna()) |  # Keep patients with no infertility diagnosis (control group)
    (data['Infertility '] > data['Psoriasis'])  # Keep patients with infertility diagnosed after psoriasis
]

In [32]:
data

,date_of_birth,GroupName,age,sex,Start_D,DiagYear,Infertility,Psoriasis,Tuberculosis,Tuberculosis s/p,...,OncBone,OncSaracoma,OncGenitalia,Oncmyeloma,OncPolycythemiaVera,OncMyelodysplastic,OncLympholiferative,OncNeurofibromatosis,OncOther,OncUnKnow
0,1906-05-15 00:00:00,Psoriasis,98,M,2005-08-03,2005.0,NaT,2005-08-03,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1907-06-15 00:00:00,Psoriasis,95,M,2003-10-12,2003.0,NaT,2003-10-12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1909-01-01 00:00:00,Psoriasis,100,M,2009-06-30,2009.0,NaT,2009-06-30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1909-05-01 00:00:00,Psoriasis,93,M,2003-04-24,2003.0,NaT,1999-11-16,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1909-06-09 00:00:00,Psoriasis,94,M,2004-09-20,2004.0,NaT,2004-02-23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138674,2016-03-15 00:00:00,Psoriasis,0,M,2017-03-14,2017.0,NaT,2017-03-14,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138676,2016-01-29 00:00:00,Psoriasis,1,M,2017-01-02,2017.0,NaT,2017-01-02,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138679,2016-01-30 00:00:00,Psoriasis,1,M,2017-03-30,2017.0,NaT,2017-03-30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138681,2016-06-15 00:00:00,Psoriasis,0,M,2017-02-28,2017.0,NaT,2017-02-28,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Identify comorbidity columns, assuming they start from "Tuberculosis" onwards
comorbidity_columns = data.columns[data.columns.get_loc("Tuberculosis"):]

# Iterate through comorbidity columns to adjust dates
for column in comorbidity_columns:
    data[column] = pd.to_datetime(data[column], errors='coerce')
    # Replace the date with NaN if it occurs after the relevant reference date
    data[column] = data.apply(
        lambda row: (
            row[column] if pd.isna(row[column]) or (
                (row['GroupName'] == 'Psoriasis' and row[column] <= row['Psoriasis']) or
                (row['GroupName'] != 'Psoriasis' and row[column] <= row['Start_D'])
            ) else pd.NaT
        ), 
        axis=1
    )

<ipython-input-33-c325cc223aff>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = pd.to_datetime(data[column], errors='coerce')
<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = data.apply(
<ipython-input-33-c325cc223aff>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = data.apply(
<ipython-input-33-c325cc223aff>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = pd.to_datetime(data[column], errors='coerce')
<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = data.apply(
<ipython-input-33-c325cc223aff>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = pd.to_datetime(data[column], errors='coerce')
<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = data.apply(
<ipython-input-33-c325cc223aff>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = pd.to_datetime(data[column], errors='coerce')
<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = data.apply(
<ipython-input-33-c325cc223aff>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = pd.to_datetime(data[column], errors='coerce')
<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

...

<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = data.apply(
<ipython-input-33-c325cc223aff>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = pd.to_datetime(data[column], errors='coerce')
<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = data.apply(
<ipython-input-33-c325cc223aff>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = pd.to_datetime(data[column], errors='coerce')
<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = data.apply(
<ipython-input-33-c325cc223aff>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = pd.to_datetime(data[column], errors='coerce')
<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = data.apply(
<ipython-input-33-c325cc223aff>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = pd.to_datetime(data[column], errors='coerce')
<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = data.apply(
<ipython-input-33-c325cc223aff>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = pd.to_datetime(data[column], errors='coerce')
<ipython-input-33-c325cc223aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [34]:
# Create binary variable for presence of infertility (Infertility_Status)
data['Infertility_Status'] = np.where(data['Infertility '].notna(), 1, 0)

# List of comorbidity columns to include in the logistic regression
comorbidity_columns = data.columns[data.columns.get_loc("Tuberculosis"):]

# To store the results of each regression
results = []

# Loop through each comorbidity to perform logistic regression
for comorbidity in comorbidity_columns:
    # Create binary variable for presence of comorbidity
    data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)

    # Check for variability: skip if there is no or minimal variation
    count = data[comorbidity + '_status'].sum()
    if count == 0 or count == len(data) or count < 5:  # Skipping if too few positive cases
        continue

    # Define the independent variable (comorbidity) and the dependent variable (Infertility_Status)
    X = data[[comorbidity + '_status']]
    y = data['Infertility_Status']

    # Add an intercept to the independent variable
    X = sm.add_constant(X)

    try:
        # Fit the logistic regression model
        model = sm.Logit(y, X)
        result = model.fit(disp=False)

        # Calculate Odds Ratio for the comorbidity
        odds_ratio = np.exp(result.params[comorbidity + '_status'])
        ci = result.conf_int().loc[comorbidity + '_status']
        ci_odds_ratio = np.exp(ci)

        # Extract p-value for the comorbidity
        p_value = result.pvalues[comorbidity + '_status']

        # Append the results for the comorbidity
        results.append({
            'Comorbidity': comorbidity,
            'Odds Ratio': odds_ratio,
            '95% CI Lower': ci_odds_ratio[0],
            '95% CI Upper': ci_odds_ratio[1],
            'p-value': p_value
        })

    except np.linalg.LinAlgError:
        # Handle any singular matrix errors due to collinearity or insufficient data
        print(f"Skipping {comorbidity} due to singular matrix issue.")
        continue

# Convert the results to a DataFrame for better viewing
results_df = pd.DataFrame(results)

# Filter for statistically significant results at the 0.05 level
significant_results = results_df[results_df['p-value'] < 0.05]

<ipython-input-34-a5df3dd86adc>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['Infertility_Status'] = np.where(data['Infertility '].notna(), 1, 0)
<ipython-input-34-a5df3dd86adc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Infertility_Status'] = np.where(data['Infertility '].notna(), 1, 0)
<ipython-input-34-a5df3dd86adc>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns a

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))
<ipython-input-34-a5df3dd86adc>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)
<ipython-input-34-a5df3dd86adc>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

Skipping Benign Brain Tumor due to singular matrix issue.


/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-34-a5df3dd86adc>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)
<ipython-input-34-a5df3dd86adc>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

Skipping Cystic Fibrosis due to singular matrix issue.


...

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-34-a5df3dd86adc>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)
<ipython-input-34-a5df3dd86adc>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-34-a5df3dd86adc>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)
<ipython-input-34-a5df3dd86adc>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-34-a5df3dd86adc>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)
<ipython-input-34-a5df3dd86adc>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

Skipping OncUnKnow due to singular matrix issue.


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-34-a5df3dd86adc>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)
<ipython-input-34-a5df3dd86adc>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)
/opt/anaconda3/lib/python3.8/site-packages/statsm

In [35]:
significant_results

,Comorbidity,Odds Ratio,95% CI Lower,95% CI Upper,p-value
4,Hepatitis C Carrier,4.957167,1.561456,15.737560,0.006607
14,Obesity,0.343501,0.139182,0.847760,0.020434
15,Hyperlipidemia,0.219886,0.106199,0.455277,0.000045
16,Hyperprolactinemia,15.290344,2.090599,111.831414,0.007223
23,Smoking,0.507559,0.307692,0.837254,0.007918
38,IHD,0.183064,0.045030,0.744219,0.017654
43,Hypertension,0.211486,0.085696,0.521915,0.000750
74,Arthropathy,0.273679,0.110894,0.675417,0.004933
85,Congenital Anomalies,6.698079,2.107377,21.289149,0.001267


In [36]:
# Create binary variable for presence of infertility (Infertility_Status)
data['Infertility_Status'] = np.where(data['Infertility '].notna(), 1, 0)

# List of comorbidity columns to include in the logistic regression
comorbidity_columns = data.columns[data.columns.get_loc("Tuberculosis"):]

# To store the results of each regression
results = []

# Loop through each comorbidity to perform logistic regression
for comorbidity in comorbidity_columns:
    # Create binary variable for presence of comorbidity
    data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)

    # Check for variability: skip if there is no or minimal variation
    count = data[comorbidity + '_status'].sum()
    if count == 0 or count == len(data) or count < 5:  # Skipping if too few positive cases
        continue

    # Define the independent variable (comorbidity) and the dependent variable (Infertility_Status)
    X = data[[comorbidity + '_status']]
    y = data['Infertility_Status']

    # Add an intercept to the independent variable
    X = sm.add_constant(X)

    try:
        # Fit the logistic regression model
        model = sm.Logit(y, X)
        result = model.fit(disp=False)

        # Calculate Odds Ratio for the comorbidity
        odds_ratio = np.exp(result.params[comorbidity + '_status'])
        ci = result.conf_int().loc[comorbidity + '_status']
        ci_odds_ratio = np.exp(ci)

        # Extract p-value for the comorbidity
        p_value = result.pvalues[comorbidity + '_status']

        # Append the results for the comorbidity
        results.append({
            'Comorbidity': comorbidity,
            'Odds Ratio': odds_ratio,
            '95% CI Lower': ci_odds_ratio[0],
            '95% CI Upper': ci_odds_ratio[1],
            'p-value': p_value
        })

    except np.linalg.LinAlgError:
        # Handle any singular matrix errors due to collinearity or insufficient data
        print(f"Skipping {comorbidity} due to singular matrix issue.")
        continue

# Convert the results to a DataFrame for better viewing
results_df = pd.DataFrame(results)

# Filter for statistically significant results at the 0.05 level
significant_results = results_df[results_df['p-value'] < 0.001]

<ipython-input-36-07ef6cffcd87>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Infertility_Status'] = np.where(data['Infertility '].notna(), 1, 0)
<ipython-input-36-07ef6cffcd87>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-36-07ef6cffcd87>:13:

Skipping Benign Brain Tumor due to singular matrix issue.


<ipython-input-36-07ef6cffcd87>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)
<ipython-input-36-07ef6cffcd87>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimiza

Skipping Cystic Fibrosis due to singular matrix issue.


/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-36-07ef6cffcd87>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimizat

...

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-36-07ef6cffcd87>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimizat

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-36-07ef6cffcd87>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)
<ipython-input-36-07ef6cffcd87>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-36-07ef6cffcd87>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-36-07ef6cffcd87>:13: Set

Skipping OncUnKnow due to singular matrix issue.


<ipython-input-36-07ef6cffcd87>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)
<ipython-input-36-07ef6cffcd87>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[comorbidity + '_status'] = data[comorbidity].notna().astype(int)
<ipython-input-36-07ef6cffcd87>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

In [37]:
significant_results

,Comorbidity,Odds Ratio,95% CI Lower,95% CI Upper,p-value
15,Hyperlipidemia,0.219886,0.106199,0.455277,0.000045
43,Hypertension,0.211486,0.085696,0.521915,0.000750


In [39]:
data = df[selected_columns]
data = data[data.GroupName == 'Psoriasis']
data = data[data.sex == 'M']
# Converting date columns to datetime for comparison
data['Psoriasis'] = pd.to_datetime(data['Psoriasis'], errors='coerce')
data['Start_D'] = pd.to_datetime(data['Start_D'], errors='coerce')

# Set Psoriasis date based on group and availability
# If 'Psoriasis' is NaT and the group is "Psoriasis", use 'Start_D' as the proxy
data['Psoriasis'] = data.apply(
    lambda row: row['Start_D'] if pd.isna(row['Psoriasis']) and row['GroupName'] == 'Psoriasis' else row['Psoriasis'],
    axis=1
)
# Convert Infertility column to datetime for comparison
data['Infertility '] = pd.to_datetime(data['Infertility '], errors='coerce')

# Filter patients who have infertility diagnosed after psoriasis or have NaN under infertility
data = data[
    (data['Infertility '].isna()) |  # Keep patients with no infertility diagnosis (control group)
    (data['Infertility '] > data['Psoriasis'])  # Keep patients with infertility diagnosed after psoriasis
]
# Identify comorbidity columns, assuming they start from "Tuberculosis" onwards
comorbidity_columns = data.columns[data.columns.get_loc("Tuberculosis"):]

# Iterate through comorbidity columns to adjust dates
for column in comorbidity_columns:
    data[column] = pd.to_datetime(data[column], errors='coerce')
    # Replace the date with NaN if it occurs after the relevant reference date
    data[column] = data.apply(
        lambda row: (
            row[column] if pd.isna(row[column]) or (
                (row['GroupName'] == 'Psoriasis' and row[column] <= row['Psoriasis']) or
                (row['GroupName'] != 'Psoriasis' and row[column] <= row['Start_D'])
            ) else pd.NaT
        ), 
        axis=1
    )

In [40]:
data

,date_of_birth,GroupName,age,sex,Start_D,DiagYear,Infertility,Psoriasis,Tuberculosis,Tuberculosis s/p,...,OncBone,OncSaracoma,OncGenitalia,Oncmyeloma,OncPolycythemiaVera,OncMyelodysplastic,OncLympholiferative,OncNeurofibromatosis,OncOther,OncUnKnow
0,1906-05-15 00:00:00,Psoriasis,98,M,2005-08-03,2005.0,NaT,2005-08-03,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,1907-06-15 00:00:00,Psoriasis,95,M,2003-10-12,2003.0,NaT,2003-10-12,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,1909-01-01 00:00:00,Psoriasis,100,M,2009-06-30,2009.0,NaT,2009-06-30,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
7,1909-05-01 00:00:00,Psoriasis,93,M,2003-04-24,2003.0,NaT,1999-11-16,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
8,1909-06-09 00:00:00,Psoriasis,94,M,2004-09-20,2004.0,NaT,2004-02-23,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138674,2016-03-15 00:00:00,Psoriasis,0,M,2017-03-14,2017.0,NaT,2017-03-14,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
138676,2016-01-29 00:00:00,Psoriasis,1,M,2017-01-02,2017.0,NaT,2017-01-02,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
138679,2016-01-30 00:00:00,Psoriasis,1,M,2017-03-30,2017.0,NaT,2017-03-30,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
138681,2016-06-15 00:00:00,Psoriasis,0,M,2017-02-28,2017.0,NaT,2017-02-28,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [41]:
#One-Hot Encoding
comorbidity_cols = data.loc[:,'Tuberculosis':].columns
data_encoded = data[comorbidity_cols].astype(str)
data_ohe = pd.get_dummies(data_encoded, columns=comorbidity_cols, prefix_sep='_')

In [ ]:
import prince

for n in range(10,80,10):
    mca = prince.MCA(n_components=n, random_state=42)
    data_mca = mca.fit_transform(data_ohe)
    
    silhouette = {}
    print(f"Silhouette Scores with MCA number of components of {n}")
    for n_clusters in range(2, 10):
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        silhouette[n_clusters] = silhouette_score(data_mca, kmeans.fit_predict(data_mca))
        
        
    print(silhouette)
    print("\n\n")